In [1]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.4.0                         
Location         d:\ner-sports-task\env\lib\site-packages\spacy
Platform         Windows-10-10.0.22621-SP0     
Python version   3.8.0                         
Pipelines                                      



In [ ]:
#!python -m spacy download en_core_web_lg

^C


DEPRECATION: https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-3.4.1/en_core_web_lg-3.4.1-py3-none-any.whl#egg=en_core_web_lg==3.4.1 contains an egg fragment with a non-PEP 508 name pip 25.0 will enforce this behaviour change. A possible replacement is to use the req @ url syntax, and remove the egg fragment. Discussion can be found at https://github.com/pypa/pip/issues/11617


     ---------------------------------------- 0.0/587.7 MB ? eta -:--:--
     -------------------------------------- 0.0/587.7 MB 960.0 kB/s eta 0:10:13
     ---------------------------------------- 0.1/587.7 MB 1.7 MB/s eta 0:05:45
     ---------------------------------------- 0.5/587.7 MB 3.3 MB/s eta 0:03:00
     ---------------------------------------- 0.8/587.7 MB 4.4 MB/s eta 0:02:14
     ---------------------------------------- 0.8/587.7 MB 4.4 MB/s eta 0:02:14
     ---------------------------------------- 1.0/587.7 MB 3.5 MB/s eta 0:02:49
     ---------------------------------------- 1.1/587.7 MB 3.7 MB/s eta 0:02:40
     ---------------------------------------- 1.4/587.7 MB 3.8 MB/s eta 0:02:34
     ---------------------------------------- 1.6/587.7 MB 3.8 MB/s eta 0:02:33
     ---------------------------------------- 1.7/587.7 MB 3.7 MB/s eta 0:02:40
     ---------------------------------------- 2.0/587.7 MB 3.8 MB/s eta 0:02:34
     ---------------------------------------- 2

In [2]:
# Importing libraries

import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import re

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

In [4]:
import json

# Load the training data (json file)

with open('D:/ner-sports-task/train_annotations.json') as fp:
  train_data = json.load(fp)

In [5]:
train_data

{'classes': ['ORG_YEAR', 'SPORTS_NAME', 'WINNING_POSITION'],
 'annotations': [['Raleigh Flynn, your outstanding performance in the Equestrian Dressage during the 2013 competition has earned you a well-deserved 1st Position. Your sportsmanship and zeal for the game are appreciated. Congratulations!\r',
   {'entities': [[51, 70, 'SPORTS_NAME'],
     [82, 86, 'ORG_YEAR'],
     [130, 133, 'WINNING_POSITION']]}],
  ['Clifford Hall, your outstanding performance in the Boxing during the 2013 competition has earned you a well-deserved 2nd Position. Your sportsmanship and zeal for the game are appreciated. Congratulations!\r',
   {'entities': [[51, 57, 'SPORTS_NAME'],
     [69, 73, 'ORG_YEAR'],
     [117, 120, 'WINNING_POSITION']]}],
  ['In recognition of their exceptional performance in the Rowing, Clifton Hamilton has been awarded the 2nd Position in the 2009 competition. We extend our hearty congratulations and best wishes for all future endeavors!\r',
   {'entities': [[55, 61, 'SPORTS_NAME'

In [6]:
# Load the Validation data (json file)

with open('D:/ner-sports-task/test_annotations.json') as fp:
  validation_data = json.load(fp)

In [7]:
validation_data

{'classes': ['ORG_YEAR', 'SPORTS_NAME', 'WINNING_POSITION'],
 'annotations': [['We take immense pleasure in presenting this certificate to Dexter Steele, who has made a remarkable achievement by securing the 1st Position in the Baseball competition, 2012. May you continue to rise and shine!\r',
   {'entities': [[128, 131, 'WINNING_POSITION'],
     [148, 156, 'SPORTS_NAME'],
     [170, 175, 'ORG_YEAR']]}],
  ['In recognition of their exceptional performance in the Beach Volleyball, Marshall Barnes has been awarded the 2nd Position in the 2009 competition. We extend our hearty congratulations and best wishes for all future endeavors!\r',
   {'entities': [[61, 71, 'SPORTS_NAME'],
     [110, 113, 'WINNING_POSITION'],
     [130, 134, 'ORG_YEAR']]}],
  ['Clifford Lamb, your outstanding performance in the Table Tennis during the 2013 competition has earned you a well-deserved 3rd Position. Your sportsmanship and zeal for the game are appreciated. Congratulations!\r',
   {'entities': [[51, 63,

In [8]:
# Converting train data into .spacy format

for text, annot in tqdm(train_data['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./training_data.spacy") # save the docbin object

100%|██████████| 268/268 [00:00<00:00, 5153.44it/s]


In [9]:
# Converting test data into .spacy format

for text, annot in tqdm(validation_data['annotations']): 
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in annot["entities"]:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents 
    db.add(doc)

db.to_disk("./validation_data.spacy") # save the docbin object

100%|██████████| 30/30 [00:00<00:00, 5003.54it/s]


In [13]:
# Getting the config file for training

!python -m spacy init config config.cfg --lang en --pipeline ner --optimize accuracy

⚠ To generate a more effective transformer-based config (GPU-only), install the
spacy-transformers package and re-run this command. The config generated now
does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: accuracy
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
# Training
# import os 

# os.system("python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./validation_data.spacy")

!python -m spacy train ./config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./validation_data.spacy

In [ ]:
# Loading best model

nlp_ner = spacy.load("D:/ner-sports-task/research/model-best") 

In [ ]:
text = "In recognition of their exceptional performance in the Artistic Swimming, Andrew Peterson has been awarded the 2nd Position in the 2009 competition. We extend our hearty congratulations and best wishes for all future endeavors!"

In [ ]:
doc = nlp_ner(text)

In [ ]:
spacy.displacy.render(doc, style="ent", jupyter=True)